In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

# ---------------------------------------------------------
# 1. 解决路径问题：将当前目录及子目录加入 Python 搜索路径
# ---------------------------------------------------------
# 获取当前 notebook 所在的绝对路径
current_dir = os.getcwd()

# 将根目录及其子模块目录加入 sys.path
# 这样你就可以直接 import config, PPO, factor 等
sys.path.append(current_dir)
sys.path.append(os.path.join(current_dir, 'config'))
sys.path.append(os.path.join(current_dir, 'factor'))
sys.path.append(os.path.join(current_dir, 'model'))
sys.path.append(os.path.join(current_dir, 'utils'))
sys.path.append(os.path.join(current_dir, 'backtest'))
sys.path.append(os.path.join(current_dir, 'PPO'))

# ---------------------------------------------------------
# 2. 导入你的项目模块
# ---------------------------------------------------------
from config import TrainingConfig
from miner_new import OptimizedSynergisticFactorMiner
from backtest import MinerBacktester
from utils import setup_logging

# 设置日志显示
logger = setup_logging(level=logging.INFO)

print("✅ 环境设置完成，模块导入成功！")

In [ ]:
# ---------------------------------------------------------
# 准备数据：这里生成一段模拟的加密货币分钟线数据
# 如果你有真实数据，请替换为 pd.read_csv('your_data.csv')
# ---------------------------------------------------------

def generate_dummy_data(n_rows=10000):  # 增加数据量
    dates = pd.date_range(start='2023-01-01', periods=n_rows, freq='15T')
    
    np.random.seed(42)
    
    # 添加可预测的趋势和动量信号
    trend = np.sin(np.linspace(0, 10*np.pi, n_rows)) * 0.002
    momentum = np.concatenate([
        np.cumsum(np.random.normal(0, 0.001, n_rows//2)),
        np.cumsum(np.random.normal(0, 0.001, n_rows//2))
    ])
    noise = np.random.normal(0, 0.001, n_rows)
    
    returns = trend + momentum * 0.0005 + noise
    price = 10000 * np.exp(np.cumsum(returns))
    
    df = pd.DataFrame(index=dates)
    df['close'] = price
    # 模拟 High/Low/Open/Volume
    df['high'] = df['close'] * (1 + np.abs(np.random.normal(0, 0.001, n_rows)))
    df['low'] = df['close'] * (1 - np.abs(np.random.normal(0, 0.001, n_rows)))
    df['open'] = df['close'].shift(1).fillna(df['close'])
    df['volume'] = np.random.randint(100, 10000, n_rows).astype(float)
    
    return df

# 1. 加载数据
print("⏳ 正在生成/加载数据...")
data = generate_dummy_data(5000) 
# data = pd.read_csv('your_data.csv', parse_dates=['timestamp'], index_col='timestamp')

print(f"✅ 数据加载完成，共 {len(data)} 条 K 线")
print(data.tail())

In [ ]:
# ---------------------------------------------------------
# 配置训练参数
# ---------------------------------------------------------
config = TrainingConfig()

# --- 根据 Notebook 运行环境进行微调 ---
config.ppo_epochs = 4           # PPO 每次更新的轮数
config.batch_size = 64          # 批次大小
config.lr_actor = 3e-4         # 学习率
config.combiner_type = 'linear' # 'linear' (线性组合) 或 'lstm' (神经网络组合)

# 挖掘参数
BATCH_SIZE = 16                 # 每次生成的公式数量
TRAIN_INTERVAL = 100             # 每多少步训练一次 PPO

print(f"⚙️ 配置已加载: {config.combiner_type} 模式")

In [4]:
# ---------------------------------------------------------
# 初始化因子挖掘器
# ---------------------------------------------------------
miner = OptimizedSynergisticFactorMiner(
    data=data,
    target_col='future_return', # 目标列名
    config=config,
    max_factors=100,             # 因子池最大保留数量
    max_expr_len=50             # 因子公式最大长度
)

print("🤖 挖掘器初始化完成，准备开始挖掘...")

2025-11-20 15:57:55,223 - INFO - Synergy Evaluator initialized (Target: Incremental Sharpe)
2025-11-20 15:57:55,224 - INFO - FactorMinerCore initialized:
2025-11-20 15:57:55,224 - INFO -   Device: cpu
2025-11-20 15:57:55,224 - INFO -   Combiner type: linear
2025-11-20 15:57:55,225 - INFO -   Features: 5
2025-11-20 15:57:55,225 - INFO -   Vocab size: 55
2025-11-20 15:57:55,225 - INFO - OptimizedSynergisticFactorMiner initialized (using refactored modules)


🤖 挖掘器初始化完成，准备开始挖掘...


In [ ]:
# ---------------------------------------------------------
# 开始运行挖掘循环
# ---------------------------------------------------------
import time
start_time = time.time()

results = miner.mine_factors(
    n_iterations=1000,
    batch_size=16,
    train_interval=TRAIN_INTERVAL,
    print_interval=5,           # 每5次打印一次进度
    early_stop_patience=1000      # 早停耐心值
)

end_time = time.time()
print(f"\n🎉 挖掘完成！耗时: {end_time - start_time:.2f} 秒")

2025-11-20 15:57:55,238 - INFO - Starting factor mining:
2025-11-20 15:57:55,241 - INFO -   Iterations: 10000
2025-11-20 15:57:55,242 - INFO -   Batch size: 16
2025-11-20 15:57:55,243 - INFO -   Early stop patience: 1000
2025-11-20 15:57:56,975 - INFO - ✨ New best VAL score: 5.0000 at iteration 0
2025-11-20 15:57:57,141 - ERROR - Combiner trial failed: array must not contain infs or NaNs
2025-11-20 15:57:58,612 - INFO - Iteration 5/10000
2025-11-20 15:57:58,612 - INFO -   Avg Reward: -4.1633
2025-11-20 15:57:58,612 - INFO -   Best VAL: 5.0000
2025-11-20 15:57:58,612 - INFO -   Pool Size: 1
2025-11-20 15:58:06,907 - INFO - Iteration 10/10000
2025-11-20 15:58:06,907 - INFO -   Avg Reward: -3.6439
2025-11-20 15:58:06,907 - INFO -   Best VAL: 5.0000
2025-11-20 15:58:06,908 - INFO -   Pool Size: 1
2025-11-20 15:58:14,931 - INFO - Iteration 15/10000
2025-11-20 15:58:14,931 - INFO -   Avg Reward: -3.7701
2025-11-20 15:58:14,931 - INFO -   Best VAL: 5.0000
2025-11-20 15:58:14,931 - INFO -   Po

KeyboardInterrupt: 

In [ ]:
# ---------------------------------------------------------
# 绘制训练历史曲线
# ---------------------------------------------------------
%matplotlib inline

print("📊 正在绘制训练历史...")
miner.plot_training_history()

# 分析性能衰退情况
miner.analyze_performance_degradation()

In [ ]:
# ---------------------------------------------------------
# 对最佳因子组合进行回测
# ---------------------------------------------------------

# 创建回测器
backtester = MinerBacktester(miner)

# 在测试集上运行回测并画图
print("\n📈 开始测试集回测...")
test_results = backtester.run(
    data_split='test', 
    top_n=5, 
    save_path='test_backtest_result.png'
)

# 打印详细的单因子表现
print("\n📋 最佳单因子表现 (Top 5):")
individual_res = backtester.get_individual_results('test')
if individual_res:
    sorted_factors = sorted(individual_res.items(), key=lambda x: x[1]['sharpe_ratio'], reverse=True)[:5]
    for name, metrics in sorted_factors:
        print(f"  {name}: Sharpe={metrics['sharpe_ratio']:.2f}, IC={metrics['ic']:.4f}")